In [1]:
from dataclasses import dataclass
from typing import *
from classes import AssociatedType, Supports, typeclass

# What is a typeclass?

**What kind of a thing is it? What is it *for*?**

A typeclass is a mechanism for capturing *ad-hoc polymorphism*: the idea of a function (or collection of them) whose implementation in general depends on the type it is (they are) applied to.
(We must have some motivation for considering all of these distinct functions "the same" in a way that is at least partially captured by what we can express, encourage, and enforce.) 
We say that all these distinct implementations are *instances* of the typeclass.
An instance is usually 'indexed' or looked up by the type of some value, and can usually be uniquely determined from that in any given context.

The rough object-oriented analogue for this is an *interface*.

For example, one might have two distinct implementations of some basic datatype adapted for different use cases, or two domain-specific mathematical stuctures that each satisfy a set of criteria and have a set of precise parrallels with respect to each other as a result.

```[python]
from classes import AssociatedType, Supports, typeclass

# `CanSpeak` is a typeclass
class CanSpeak(AssociatedType):
  '''Implementation-independent specification'''

# `CanSpeak` instances must define this function 
# with this type signatue (or something close).
@typeclass(CanSpeak)
def speak(instance: "Supports[CanSpeak]") -> str:
  '''Retuns string with "speech".'''
# no implementation!  


@dataclass
class Dog(Animal):
  name: str

@speak.instance(Dog)
def speakInstance(instance: Dog) -> str:
  return "woof!"

@dataclass
class SmartToaster(Robot):
  heat_level: int
  ads_to_play: list[Ad]
  
@speak.instance(SmartToaster)
def anotherSpeakInstance(instance: SmartToaster) -> str:
  return next(ads_to_play)
  
speak(Dog("Fido"))
speak(SmartToaster(11, theSameAdButFiveTimesInArow))
```




Typeclasses are also a limited mechanism for "semantic metaprogramming": a typeclass is a function fom types to values, and those values are functions - code!
  - when you call `speak(Dog("Fido"))`, you're not making any overt reference to a particular implementation, so it must be inferred.
    - In languages with first-class support for typeclasses, there are mechanisms for allowing the (at least compile-time) *context of the call site* to a typeclass's function to determine how to resolve which typeclass instance to use, in case there are several available. 

 **What is it?**

 - A typeclass $t$ is mainly defined by an explicit set of required functions.
   - These functions may have a default implementation.
 - For any type to be an *instance* of the type class, it must have an implementation for every function associated with the typeclass.
 - Typeclasses can also be *constrained* or *refined*, so that only types that are subclasses (or instances of some othe typeclass) can be instances.
 - The most commonly used typeclasses all have strong associated expectations about how all the required functions behave together (above and beyond their type signatures), often forming loose networks and hierarchies.
   - `Order.py` is typical in presenting an algebraically motivated hierarchy of typeclasses, starting at an arbitrary level that seems useful for enough applications.

Per the documentation of the `classes` package, a typeclass is a very simple idea, and a compelling alternative to the half-baked, bolted-on grabbag of object-oriented mechanisms for polymorphism available by default in Python (subtyping, abstract classes, protocols, single dispatch...).
 - See https://sobolevn.me/2021/06/typeclasses-in-python
 - Or https://classes.readthedocs.io/en/latest/index.html

### Why use them? Why here?

Python's underlying type and class abstractions are an evolving and maturing mess with some limitations on how much they can really achieve or be built on without a new language specification.
Why try typeclasses?

*Typeclasses are the right tool for the job.*

The scope of this project is small and essentially about 
 - presenting different implementations of the same few datatypes that all represent mathematical structures.
 - testing/demonstrating that expected mathematical properties hold of these implementations.
 
Typeclassess that have associated *algebaic laws* describing the expected behavior of the functions required to be implemented are some of the most common and foundational uses of typeclasses in languages that have them.

#### Why not use subtyping?

Without rehashing the debate on the merits of object-oriented progamming...
 - Subtyping and (especially) inheritance make code hard to reason about.
 - Typeclasses don't require much, and don't impose a lot of baggage.
 - Typeclasses allow more code reuse than inheritance.
 - Typeclasses are one of the basic tools (dating back to the 90s) that help address [the expression problem](https://en.wikipedia.org/wiki/Expression_problem).

#### Ok, but why not use an existing Python mechanism for expressing something like the object-oriented idea of an 'interface'?

 - See https://sobolevn.me/2021/06/typeclasses-in-python
 - Or https://classes.readthedocs.io/en/latest/index.html

#### Fine. Why bother with Python's garbage type system at all?

 - Types and typeclasses still convey useful information about developer intent and the *meaning* of code.
 - Types and typeclasses make it easier to write code that is composable, reusable, and easier (or even just practically possible) to refactor instead of rewrite.
 - Ease of porting code to/from languages with better type systems.
 - Fewer bugs.
 - Fewer tests.
 - Test automation.

# The typeclass implementation here w/ examples

A *typeclass* is a garden variety class
 - that extends (directly or indirectly) `AssociatedType`
 - with one or more functions that are associated with that class via decorators.

In [2]:
# A typeclass
class CanFoo(AssociatedType):
  """Can call foo"""

  
# A function whose signatue we want to associate 
# with `CanFoo`.
# This (together with the `AssociatedType` above)
# makes `CanFoo` a typeclass.
@typeclass(CanFoo)
def foo(instance : "Supports[CanFoo]") -> None:
  """foos"""
# Notice it has no implementation.

# Below is exactly the same snippet, but with an 
# implementation.
# If comment out the snippet above, and uncomment
# the snippet below, we will get an error.
# @typeclass(CanFoo)
# def foo(instance: "Supports[CanFoo]") -> None:
#   """foos"""
#   print(f"foo @ {instance}") #


# For the time being, this is how a default instance
# is defined.
# Note to self: stay tuned for https://github.com/dry-python/classes/issues/307
@foo.instance(object)
def fooForObjects(instance: object) -> None:
  print(f"object: foo @ {instance}")


# An example of how you might define a type-indexed
# instance of the `foo` function above.
@foo.instance(int)
def _foo_int(instance: int) -> None:
  print(f"int: foo @ {instance}") 
# Because `foo` is the only "typeclass" decorated
# function associated with `CanFoo`, that means
# `int` is now an instance of the `CanFoo` typeclass.


# Below we define a custom datatype and show how to
# make an instance of the `CanFoo` typeclass
@dataclass
class IdNumber:
  value: int

@foo.instance(IdNumber)
def _foo_Id(instance: IdNumber) -> None:
  print(f"foo! @ {instance}")


foo(3)      # "int: foo @ 3"
foo("spam") # "object: foo @ spam"

an_id = IdNumber(1234)
foo(an_id)  # "foo! @ IdNumber(value=1234)"

int: foo @ 3
object: foo @ spam
foo! @ IdNumber(value=1234)


## More complicated examples

Here we have a typeclass
 - with more than one required function.
 - with one function that has no default implementation.

In [3]:
#typeclass
class CanBarBaz(AssociatedType):
  """Can call bar and baz"""

#signature of bar
@typeclass(CanBarBaz)
def bar(instance: "Supports[CanBarBaz]") -> None:
  """bars"""
  
#signature of baz
@typeclass(CanBarBaz)
def baz(instance: "Supports[CanBarBaz]") -> None:
  """bazs"""
  
# default instance for `bar`
@bar.instance(object)
def _bar_default(instance: object) -> None:
  return f"object: bar @ {instance} with {baz(instance)}"
# NOTE how the default implementation for `bar` 
# *calls* `baz` and hence depends on the existence of
# an appropriate implementation.
  
# (No default instance for 'baz'...)

Below, we define a custom type that is an instance of `CanBarBaz`:

In [4]:
@dataclass
class Participant:
  name: str

@baz.instance(Participant)
def _baz_part(instance: Participant) -> None:
  return f"baz from {instance}"

p = Participant("Grad, U")
print(bar(p))
print(baz(p))

object: bar @ Participant(name='Grad, U') with baz from Participant(name='Grad, U')
baz from Participant(name='Grad, U')


What about when you have a typeclass operation that constrains more than one type?

In [8]:
A = TypeVar('A')
B = TypeVar('B')

class Functor(Generic[A], AssociatedType):
  '''
  In a nutshell, something is an instance of a
  Functor if it is some kind of container or
  computational 'context' that has systematic,
  nicely behaved complications or other 
  consequences for 
   - function application.
   - function composition.
   
  The function that handles function application 
  and composition in light of complications or
  context posed by some functor instance F is 
  called `map` or `fmap`:
    fmap : (A -> B) x F[A] -> F[B]
  It must obey two laws:
    fmap(identity, fa) = identity(fa)
    fmap(g \of f,  fa) = fmap(g, fmap(f, fa))
    
  `fmap` takes
    - a function that doesn't know anything
      about F
    - some F structure or context containing 
      value(s) from A
  and takes care of any complications that may
  arise due to F while applying the function,
  resulting in an F structure or context containing
  a transformed B value.
  
   
  Example: The concrete starting place is *lists*.
  Given two functions
    f : int -> int
    f = lambda x: x + 2
    
    g : int -> str
    g = lambda x: str(x)
  and a list l : list[int]
    l = [10,3]
  is there a natural way to "contextually 
  redefine" function application so it works the
  way it does without having to know anything 
  about the fact that lists are involved?
  In other words, can you define a combinator 
    fmap : (A -> B) x list[A] -> list[B]
  that takes 
   - *any garden variety function* turning
  A values into B values
   - a list of A values
  and returns
   - a list of B values
  with the list structure of the input preserved
  in the output?
  Yes:
    fmap = lambda f, l: [f(x) for x in l]
  '''
  
@typeclass(Functor)
def fmap(f: A -> B, Fa: Supports[Functor]) -> Supports[Functor]:
  '''Function application in a functorial context.'''

@dataclass
class Maybe(Generic[A], Functor):
  value: A
  
# @adt


 # @typeclass(Semiring)

In [ ]:
class Magma(AssociatedType):
  '''
  A magma is a carrier set A together with
  a binary operation * such that
   - * is total over A.
   - A is closed under *.
  
  Intuitively, a magma can define binary 
  trees with data drawn from A at leaves.
  
  Examples of magma elements over A = {a,b}:
    a
    b
    a*b
    (a*b)*a
    (b*(a*b))*a
  Parentheses are not optional.
  '''

@typeclass(Magma)
def multiply(a: Supports[Magma], b: Supports[Magma]) -> Supports[Magma]:
  '''Total and closed binary operation over A.'''

#if this were a real magma typeclass, we'd specify
# other operations that make sense for magmas...

@multiply.instance(int)
def _multiply_int(a: int, b: int) -> int:
  return a * b

class Semigroup(Magma):
  '''
  A semigroup is a magma whose operation is
  associative:
   a*(b*c) = (a*b)*c
  for all a,b,c : A.
  
  Intuitively, a semigroup is a non-empty sequence.
  '''
  
@typeclass(Semigroup)
def len(a: Supports[Semigroup]) -> int:
  

class Monoid(Magma):
  '''
  A monoid is a magma with two differences:
   - The operation is associative:
       a*(b*c) == (a*b)*c
     for all a,b,c in the carrier set A.
   - There is a unique two-sided identity element e:
       e*a == a*e == a
     for all a in the carrier set.
   Intuitively, 
  '''

# @typeclass(Semigroup)
# def 
# # class Semiring(AssociatedType):
# #   '''
# #   A semiring is a carrier set A where two
# #   distinct algebraic structures come together
# #   and interact in a nice way:
# #    - (A,+,0) forms a commutative group.
# #    - (A,*,1) forms a monoid.
# #    - Multiplication distributes over addition from both sides.
# #    - $\forall a, 0 * a = a * 0 = 0$: the additive identity is the multiplicative absorbing element.
# #   '''
  
# # @typeclass(Semiring)

# # class Semiring(AssociatedType):
# #   '''
# #   A semiring is a carrier set A where two
# #   distinct algebraic structures come together
# #   and interact in a nice way:
# #    - (A,+,0) forms a commutative group.
# #    - (A,*,1) forms a monoid.
# #    - Multiplication distributes over addition from both sides.
# #    - $\forall a, 0 * a = a * 0 = 0$: the additive identity is the multiplicative absorbing element.
# #   '''
  
#

In [9]:
multiply(3, 4)

12

Below we have a function that's ostensibly defined on any x that is an instance of *both* `CanFoo` and `CanBarBaz`:

In [56]:
# An ordinary function whose implementation/semantics should not be expected to
# be dependent on the specific implementation x has for either `CanFoo` or 
# `CanBarBaz`
# YES it says union below, but the result is intersection. ¯\_(ツ)_/¯
# That is the current state of the stable branch, but that is not where it will
# be -> https://github.com/dry-python/classes/issues/206#issuecomment-867531319
def functionThatNeedsFooAndBarBaz(x: Supports[Union[CanFoo, CanBarBaz]]) -> str:
  return f"{foo(x)} | {bar(x)}"

print(functionThatNeedsFooAndBarBaz(p)) # "None | object: bar @ Participant(name='Grad, U') with baz from Participant(name='Grad, U')"

# 3 (and generally ints) has an instance for CanFoo but not for CanBarBaz
# print(foo(3)) #"int: foo @ 3\nNone"
# bar(3) # will raise a "not implemented" exception
# print(functionThatNeedsFooAndBarBaz(3)) #will raise one as well, for the same reason

object: foo @ Participant(name='Grad, U')
None | object: bar @ Participant(name='Grad, U') with baz from Participant(name='Grad, U')
int: foo @ 3
None


If there's some specific kind of behavior that occurs in something we want to capture with more than one kind of typeclass, we simply express them using the ```Supports[Union[A, ...]]``` class.

Below, we have a typeclass that is simply the intersection of some existing ones.

In [63]:
# A = TypeVar("A", bound = Supports[CanFoo, CanBarBaz])

class CanFooAndBarBaz(AssociatedType):
# class CanFooAndBarBaz(CanFoo, CanBarBaz):
  '''Fooing with now with BarBazzing'''
  
@typeclass(CanFooAndBarBaz)
def needsFooAndBarBaz(instance: Supports[Union[CanFoo, CanBarBaz]]) -> str:
  '''some function that depends on functionality from both typeclasses'''
  
# yes, it says `object`, but remember all that means is it's a default 
# implementation
@needsFooAndBarBaz.instance(object)
def tripleThreat(instance: object) -> str:
  return f"{foo(instance)} - - - {bar(instance)}"

print(needsFooAndBarBaz(p)) # "object: foo @ Participant(name='Grad, U')\nNone - - - object: bar @ Participant(name='Grad, U') with baz from Participant(name='Grad, U')"
#print(needsFooAndBarBaz(3)) # will print "int: foo @ 3" before raising a 'not implemented' exception...

object: foo @ Participant(name='Grad, U')
None - - - object: bar @ Participant(name='Grad, U') with baz from Participant(name='Grad, U')


What about creating a typeclass that inherits from another typeclass?

In [64]:
# This is possible
class CanFooSub(CanFoo):
  '''Fooing, but with modification'''
  
#if we redefine any of CanFoo's functions and we're not careful, we'll overwrite
# existing definition.
# Here for example, the foo immediately below is in the same namespace/scope
# as the earlier one (as opposed to being inside different modules, for example).
# @typeclass(CanFooSub)
# def foo(instance: Supports[CanFooSub]) -> str:
#   '''Overwrites foo!'''